In [1]:
import os
import shutil
import random
import pandas as pd

import numpy as np

import sqlite3

from sklearn.metrics import f1_score, accuracy_score, recall_score, confusion_matrix

from keras.models import Model, load_model
from keras.layers import Conv2D, Flatten, Dense, GlobalAveragePooling2D

from PIL import Image
import Augmentor

import utils
import crud

Using TensorFlow backend.
/home/fytroo/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
model_df = crud.models()
model_df['path'] = ['models/{}.hdf5'.format(name) for name in model_df['name']]
model_df

,id,name,validation_table,dataset_id,dataset_name,path
0,1,eyes_lenet1,validation_5a260ba0b037cbb5ae741317,5a260aeab037cbb588eafc87,eyes,models/eyes_lenet1.hdf5
1,2,eyes_lenet2,validation_5a260d02b037cbb6d90c3dcd,5a260aeab037cbb588eafc87,eyes,models/eyes_lenet2.hdf5
2,3,eyes_lenet3,validation_5a260dceb037cbb7ca64be27,5a260aeab037cbb588eafc87,eyes,models/eyes_lenet3.hdf5
3,4,eyes_lenet4,validation_5a260e88b037cbb8ada2d955,5a260aeab037cbb588eafc87,eyes,models/eyes_lenet4.hdf5
4,5,brain_lenet1,validation_5a260ebfb037cbb9b1df6d11,5a260aeab037cbb588eafc88,brain,models/brain_lenet1.hdf5
5,6,brain_lenet2,validation_5a260ef4b037cbba424037b0,5a260aeab037cbb588eafc88,brain,models/brain_lenet2.hdf5
6,7,brain_lenet3,validation_5a260f29b037cbbacf7a709d,5a260aeab037cbb588eafc88,brain,models/brain_lenet3.hdf5
7,8,brain_lenet4,validation_5a26106bb037cbbbef355865,5a260aeab037cbb588eafc88,brain,models/brain_lenet4.hdf5


In [3]:
def try_load_model(model_path):
    try:
        return load_model(model_path)
    except:
        return None

In [5]:
models = {}

for itr in model_df.iterrows():
    model_id = itr[1]['id']
    model = try_load_model(itr[1]['path'])
    if model:
        models.update({
            model_id: model
        })

In [6]:
models

{1: <keras.models.Sequential at 0x7f1c35dbcf98>,
 2: <keras.models.Sequential at 0x7f1c35d716a0>,
 3: <keras.models.Sequential at 0x7f1c35b38c50>,
 4: <keras.models.Sequential at 0x7f1c2f66df60>,
 5: <keras.models.Sequential at 0x7f1c2ec82a58>,
 6: <keras.models.Sequential at 0x7f1c2e06fe10>,
 7: <keras.models.Sequential at 0x7f1c2dec2400>,
 8: <keras.models.Sequential at 0x7f1c2c3a6898>}

with sqlite3.connect('curontab.db') as con:
    c = con.cursor()
    q = '''
    update model set dataset_name = 'eyes' where dataset_id= "5a24a8bab037cb1164d683c2"
    '''#.format(name='brain')
    c.execute(q)
    con.commit()    

with sqlite3.connect('curontab.db') as con:
    c = con.cursor()
    q = '''
    alter table model add column dataset_name[varcher(64)]
    '''
    c.execute(q)
    con.commit()
    

In [7]:
def judge_img_by_data_path(data_path, model_id):
    model_info = crud.model_by_id(model_id)
    dataset_name = model_info['dataset_name']
    label2id = crud.dataset_by_name(dataset_name)['label2id']
    id2label = {id:label for label, id in label2id.items()}

    model = models[model_id]
    batch, h, w, c = model.input_shape
    x = utils.img2arr(data_path, resize=(h,w), rescale=1./255)
    xs = np.array([x])

    y_preds = model.predict(xs)
    y_pred = y_preds[0]
    pred = y_pred.argmax()
    label_pred = id2label[pred]
    
    res = {
        'y_pred': y_pred.tolist(),
        'model_id': model_id,
        #'dataset_name': dataset_name,
        #'label2id': label2id,
        #'id2label': id2label,
        'path': data_path,
        'label_pred': id2label[pred],
        'pred': pred,
    }
    return res
#judge_img_by_data_path('dataset/eyes/N/148_0015.JPG', 1)

{'label_pred': 'N',
 'model_id': 1,
 'path': 'dataset/eyes/N/148_0015.JPG',
 'pred': 1,
 'y_pred': [0.4956947863101959, 0.5043052434921265]}

In [8]:
def judge_imgs_by_data_path(data_paths, model_id):
    model_info = crud.model_by_id(model_id)
    dataset_name = model_info['dataset_name']
    label2id = crud.dataset_by_name(dataset_name)['label2id']
    id2label = {id:label for label, id in label2id.items()}

    model = models[model_id]
    batch, h, w, c = model.input_shape
    xs = np.array([utils.img2arr(f, resize=(h,w), rescale=1./255) for f in data_paths])

    y_preds = model.predict(xs)
    df = pd.DataFrame()
    df['pred'] = y_preds.argmax(axis=1)
    df['label_pred'] = [ id2label[pred] for pred in df['pred']]
    df['path'] = data_paths
    df['model_id'] = model_id
    ds = list(df.T.to_dict().values())
    ds

    for d, y_pred in zip(ds, y_preds.tolist()):
        d.update({'y_pred':y_pred})
    res = ds
    return res
imgs = ['dataset/eyes/N/148_0015.JPG', 'dataset/eyes/AB/159_0003.JPG']
#judge_imgs_by_data_path(imgs, 2)

[{'label_pred': 'N',
  'model_id': 2,
  'path': 'dataset/eyes/N/148_0015.JPG',
  'pred': 1,
  'y_pred': [0.12207779288291931, 0.8779221773147583]},
 {'label_pred': 'AB',
  'model_id': 2,
  'path': 'dataset/eyes/AB/159_0003.JPG',
  'pred': 0,
  'y_pred': [0.9999796152114868, 2.035386387433391e-05]}]

In [10]:
def judge_img_by_data_id(data_id, model_id):
    data_path = crud.uploaded_data(data_id)['path']
    res = judge_img_by_data_path(data_path, model_id)
    return res
#judge_img_by_data_id(43, 4)

In [12]:
def judge_imgs_by_data_id(data_ids, model_id):
    df = crud.uploads_df()
    data_paths = df[df['id'].isin(data_ids)]['path'].values.tolist()
    res = judge_imgs_by_data_path(data_paths, model_id)
    return res
#judge_imgs_by_data_id([27,31,35], 4)